## Contextos

In [ ]:
x = 10
class A
  puts x
end
# NameError: undefined local variable or method `x' for A:Class

def m
  puts x
end
m # NameError: undefined local variable or method `x' for main:Object

## Flat scope

In [ ]:
a = 5
p = lambda {a = a + 1}
p.call # 6
p.call # 7

a   # 7

In [ ]:
x = 10
define_method(:m) do
  x + 5
end
m # 15

In [ ]:
x = 10
una_clase = Class.new do
  x += 5
end
x # 15

## Lambdas / Procs

In [ ]:
lam = lambda { |x| puts x } # creates a lambda that takes 1 argument 
lam.call(2) # prints out 2

In [ ]:
lam.call # ArgumentError: wrong number of arguments (0 for 1)

In [ ]:
lam.call(1,2,3) # ArgumentError: wrong number of arguments (3 for 1)

In [ ]:
proc = Proc.new { |x| puts x } # creates a proc that takes 1 argument 
proc.call(2) # prints out 2 
proc.call # returns nil 
proc.call(1,2,3) # prints out 1 and forgets about the extra arguments

In [ ]:
def lambda_test 
  lam = lambda { return } 
  lam.call 
  puts "Hello world"
end

lambda_test # calling lambda_test prints ‘Hello World’

In [ ]:
def proc_test 
  proc = Proc.new { return } 
  proc.call 
  puts "Hello world"
end

proc_test # calling proc_test prints nothing

## Bloques

In [ ]:
def bloque_test
  yield(3)
end

bloque_test do |x|
  x + 2
end

In [ ]:
# Cuando sea necesario, se puede pasar un proc en lugar de un bloque usando &
def bloque_proc_test(&bloque)
  bloque.call(3)
end

bloque_test do |x|
  x + 2
end

## Receptor implícito

In [ ]:
class Usuario
  attr_accessor :edad

  def initialize(edad)
    @edad = edad
  end

  def mayor_de_edad?
    edad >= 18
  end
end

Usuario.new(19).mayor_de_edad?

## Contextos y bloques

In [ ]:
class Usuario
  def lazy_edad
    proc { edad }
  end
end

Usuario.new(19).lazy_edad.call

In [ ]:
class Usuario
  def con_bloque(bloque)
    bloque.call
  end
end

mayor = Usuario.new(19)
menor = Usuario.new(15)
menor.con_bloque(mayor.lazy_edad)

In [ ]:
# Si necesito que el bloque mande mensajes a otro usuario, voy a tener que pasarlo por parámetro
class Usuario
  def edad_de
    proc { |u| u.edad }
  end
end

Usuario.new(19).edad_de.call(Usuario.new(15))

## Cambiar el contexto

In [ ]:
class Usuario
  def edad_de
    proc { edad }
  end
end

bloque = menor.edad_de

In [ ]:
mayor.instance_eval(&bloque)

In [ ]:
menor.instance_eval(&bloque)

## Ejercicio

In [ ]:
class Cocinero
  attr_accessor :ingredientes, :empanada

  def initialize
    self.ingredientes = []
  end

  def preparar_empanadas(&receta)
    self.instance_eval(&receta)
  end

  def saltear_cebolla
    self.agregar("cebolla salteada")
  end

  def cocinar_carne
    self.agregar("carne cocida")
  end

  def agregar(ingrediente)
    self.ingredientes << ingrediente
  end

  def rellenar_tapas
    self.empanada = Empanada.new(ingredientes)
  end

  def hornear
    self.empanada.hornear
    self.empanada
  end
end

In [ ]:
class Empanada
  attr_accessor :ingredientes, :horneada

  def self.receta
    proc {
      saltear_cebolla
      cocinar_carne
      agregar 'huevo'
      agregar 'aceitunas'
      rellenar_tapas
      hornear
    }
  end

  def self.tucumana
    proc {
      saltear_cebolla
      cocinar_carne
      agregar 'huevo'
      agregar 'aceitunas'
      rellenar_tapas
      agregar_papa
      hornear
    }
  end

  def initialize(ingredientes)
    self.ingredientes = ingredientes
  end

  def hornear
    self.horneada = true
  end
end

In [ ]:
maiu = Cocinero.new
maiu.preparar_empanadas(&Empanada.receta)

In [ ]:
axel = Cocinero.new
axel.define_singleton_method(:agregar) do |ingrediente|
  if ingrediente != "aceitunas"
    super(ingrediente)
  end
end
axel.preparar_empanadas(&Empanada.receta)

In [ ]:
# tucumanas
maiu.preparar_empanadas(&Empanada.tucumana) # esto rompe por la papa

## Method Missing

In [ ]:
class Cocinero
  def method_missing(nombre, *args, &bloque)
    if nombre.to_s.start_with?("agregar_")
      ingrediente = nombre.slice("agregar_".size, nombre.size)
      self.agregar(ingrediente)
    else
      super
    end
  end

  def self.respond_to_missing?(sym, priv = false)
    sym.to_s.start_with?('agregar_')
  end
end

In [ ]:
maiu.preparar_empanadas(&Empanada.tucumana) # ahora si